In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
import requests
from bs4 import BeautifulSoup
from decimal import Decimal

In [ ]:
def transform_data(html_content):
    # get the html text into a dataframe
    df = html_content_to_df(html_content)

    # clean our data within the dataframe
    df_clean = clean_data_in_df(df)

    # generate our calendar of weeks
    df_week_data = generate_week_data()

    # merge our dataframe with the week metadata
    df_final = merge_df_with_week_metadata(df_clean,df_week_data)

    return df_final

    

In [ ]:
def generate_week_data():
    tdelta_begin_wk = timedelta(days = -6)
    tdelta_next_week_end_date = timedelta(days = 7)

    season_weeks = [
        ['2017-18',datetime(2017,5, 28),0,53],
        ['2018-19',datetime(2018,6, 3),0,52],
        ['2019-20',datetime(2019,6, 2),0,41],
        ['2020-21',datetime(2020,6, 1),-1,52],
        ['2021-22',datetime(2021,5, 30),10,52],
        ['2022-23',datetime(2022,5, 29),0,52]
    ]

    week_df_columns = ['season','week_num','week_start_date','week_end_date']
    week_data = []

    for row in season_weeks:
        season = row[0]
        season_start_date_we = row[1]
        num_weeks_offset = row[2]
        week_end_date = season_start_date_we + timedelta(days=7*num_weeks_offset)
        num_weeks = row[3]

        if not(season == '2020-21'): #exclude the 2020-21 season because of Pandemic
            
            for i in range(num_weeks_offset+1,num_weeks+1):
                week_start_date = week_end_date + tdelta_begin_wk
                
                week_data.append([season,i,week_start_date,week_end_date])
    
                week_end_date += tdelta_next_week_end_date
            
    df_week_data = pd.DataFrame(week_data,columns=week_df_columns)

    return df_week_data

In [5]:
def html_content_to_df(html_content):
    soup = BeautifulSoup(html_content, 'html5lib') # oddly, html.parser was not finding the header row tr tag of the table

    data = []

    if soup: # make sure there's content to process
        # get the report week ending date
        report_we_date = soup.find(id='vault-search-results-sort-select').find('option', selected=True).text
        report_we_date_dt = datetime.strptime(report_we_date,'%Y-%m-%d')

        # get the week number
        weeknum = soup.find('div',class_='week-count').find('span').text

        # find the table element on the webpage
        table = soup.find('div', class_='vault-grosses-result').find('table')

        # find all of the rows within the html table
        rows = table.find_all('tr')

        # iterate over all of the table rows including within the table header
        for row in rows:

            row_data = []
            cells = row.find_all(['th', 'td'])
            
            # iterate over all of the cells within the row and fetch the respective cell value
            for cell in cells:
                # if it is a column header
                if cell.name=='th':
                    row_data.append(cell.a.text.strip())

                    if(cell.find('span',class_='subtext')):
                        row_data.append(cell.find('span',class_='subtext').text.strip())
                    else:
                        row_data.append(None)
                # it is not a column header
                else:
                    if(cell.find('span',class_='data-value')):
                        row_data.append(cell.find('span',class_='data-value').text.strip())
                    else:
                        row_data.append(None)

                    if(cell.find('span',class_='subtext')):
                        row_data.append(cell.find('span',class_='subtext').text.strip())
                    else:
                        row_data.append(None)
            
            # append the row data
            data.append(row_data)

        # create a dataframe from the processed data
        df = pd.DataFrame(data[1:], columns=data[0])

        # add an additional column in the dataframe for the report week ending date
        df['week_end_date'] = report_we_date_dt
        return df

    else:
        print("there is no soup")
        return False


In [3]:
def clean_data_in_df(df_to_clean):
    df = df_to_clean.copy()
    
    # drop NA columns
    df = df.dropna(axis=1, how='all')

    # add a column name for the Venue
    df.columns.values[1] = 'venue_name'

    # rename columns to what they are in the db table
    df.rename(columns={
        'Show' : 'show_name',
        'This Week Gross' : 'tw_gross',
        'Potential Gross' : 'potential_gross',
        'Diff $' : 'diff_tw_lw_gross',
        'Avg Ticket' : 'avg_ticket_price',
        'Top Ticket' : 'max_ticket_price',
        'Seats Sold' : 'seats_sold',
        'Seats in Theatre' : 'seats_in_theater',
        'Perfs' : 'performances',
        'Previews' : 'preview_performances',
        f'% Cap' : 'pct_capacity',
        f'Diff % cap' : 'diff_tw_lw_pct_capacity'
    },inplace=True)

    # convert the currency strings to floats
    def convert_currency_to_decimal(value):
        # Check if the value is a string and contains a dollar sign
        if isinstance(value, str) and '$' in value:
            # Remove dollar sign and comma, and convert to float
            return Decimal(value.replace(',', '').replace('$', ''))
        return value

    currency_columns_to_convert = ['This Week Gross','Potential Gross','Diff $','Avg Ticket','Top Ticket']
    df[currency_columns_to_convert] = df[currency_columns_to_convert].applymap(convert_currency_to_decimal)

    # convert string numbers to integers
    def convert_str_numbers_to_numbers(value):
        converted_value = value
        if isinstance(value, str):
            if ',' in value:
                converted_value = converted_value.replace(',','')
            return int(converted_value)
        return converted_value

    number_columns_to_convert = ['Seats Sold','Seats in Theatre','Perfs','Previews']
    df[number_columns_to_convert] = df[number_columns_to_convert].applymap(convert_str_numbers_to_numbers)

    # convert percent values to floats
    def convert_pct_to_float(value):
        if isinstance(value, str) and '%' in value:
            return float(value.replace('%', '')) / 100
        return value

    pct_columns_to_convert = ['pct_capacity','diff_tw_lw_pct_capacity']
    df[pct_columns_to_convert] = df[pct_columns_to_convert].applymap(convert_pct_to_float)

    return df

In [ ]:
def merge_df_with_week_metadata(df,df_week_data):
    df_merged = pd.merge(df,df_week_data,how='left',on='week_end_date')

    df_final = df_merged[[
        'show_name',
        'venue_name',
        'season',
        'week_num',
        'week_start_date',
        'week_end_date',
        'tw_gross',
        'potential_gross',
        'diff_tw_lw_gross',
        'avg_ticket_price',
        'max_ticket_price',
        'seats_sold	seats_in_theater',
        'performances',
        'preview_performances',
        'pct_capacity',
        'diff_tw_lw_pct_capacity'
    ]]

    return df_final
